In [90]:
# setup
import warnings;
warnings.filterwarnings('ignore'); #tensorflow gives me weird stuff
import numpy as np;
import tensorflow as tf;
from numpy import matmul as mul
from numpy.linalg import norm as norm
from scipy import sparse
from tensorflow.sparse import to_dense
tf.enable_eager_execution()

In [91]:
#Hadamard product

def hadamard(matrices, skip_matrix = -1):
    #matrices is a list of numpy ndarrays (shape of all matrices must be the same)
    shp = matrices[0].shape
    #Commented out for performance increase
#     for m in matrices: 
#         if m.shape != shp:
#             return None
    ret = np.ones(shp)
    for i in range(shp[0]):
        for j in range(shp[1]):
            for num,k in enumerate(matrices):
                if(num != skip_matrix):
                    ret[i][j] *= k[i][j]
                    if ret[i][j] == 0: #just a lil optimization
                        break;
    return ret

#MTTKRP helper methods

def calc_indices(t, dims, n):
    #t is the target row we want (zero indexed)
    #dims is the dimensions of the tensor
    #n is the factor matrix we dont want to include
    
    #first get cumulative products on our factors
    prods = [1]
    nd = len(dims)
    for i in range(nd-1,-1,-1):
        if i!=n:
            p = prods[-1] * dims[i]
            prods.append(p)
    prods.reverse()
    prods = prods[1:]
    ret = [] #the coefficients of the thingy we want
    for i in range(nd - 1):
        n = t // prods[i];
        ret.append(n)
        t -= (prods[i] * n)
    return ret

def khatri_rao_at_ij(factors, i, j, dims, n):
    #dims is the list of dimensions, factors is the actual factor matrix, 
    #factors is the list of all factor matrices
    #gives you the khatri rao product at a certain index
    product = 1
    i_vals = calc_indices(i, dims, n)
    for num, factor in enumerate(factors):
        if num == n:
            pass
        if num < n:
            product *= factors[num][i_vals[num]][j]
        if num > n:
            product *= factors[num][i_vals[num-1]][j]
    return product

def find_index_unfolded(dims, n, idx):
    #finds the coordinates of the unfolded tensor element given the coordinates of the not-unfolded tensor
    j = 0;
    nd = len(dims)
    prod = 1;
    
    for i in range(nd):
        if i != n:
            j += idx[i] * prod
            prod *= dims[i]
    
    return (idx[n], j)

def MTTKRP(X, factors, weights, n,rank,dims):
    # Matricized Tensor Times Khatri Rao Product
    # X unfolded is short and fat, khatri-rao is tall and skinny
    # output is short and skinny
    
    nd = len(dims)
    output = np.zeros((dims[n],rank))
    indices = X.indices.numpy()
    values = X.values.numpy()
    
    for l in range(len(values)):
        cur_index = indices[l]
        cur_value = values[l]
        i,j = find_index_unfolded(dims, n, cur_index) 
        # gets the index of this element if we did a mode n unfolding of X
        
        for r in range(rank):
            output[i][r] += cur_value * khatri_rao_at_ij(factors,j,r,dims,n) * weights[0][r]
            
    return output

def cp_als(X, rank, iterations = 3):
    
    dims = X.shape.as_list()
    nd = len(dims)
    factors = [np.random.random((d,rank)) for d in dims]
    l = np.ones((1,rank))
    
    for iteration in range(iterations): 
        for n in range(nd):
            to_be_hadamarded = [mul(f.T,f) for f in factors] + [mul(l.T,l)]
            v = hadamard(to_be_hadamarded, skip_matrix=n)   
            vinv = np.linalg.pinv(v)
            mk = MTTKRP(X, factors, l, n, rank, dims)
            An = mul(mk,vinv)
            for i in range(rank):
                col = An[:,i]
                weight = norm(col)
                # print( " weight: ", weight, 'col: ', col)
                if(abs(weight)> 0.000001):
                    An[:,i]/=weight
                    l[0][i]*=weight
            factors[n] = An
            
    return l, factors

In [92]:
def expand(factors, weights, dim_no, cur_idx, cur_prod, all_vals, rank):
    #this method just writes to all values, so all values needs to be saved somewhere
    if dim_no == len(factors):
        value = 0;
        for r in range(rank):
            value += cur_prod[r] * weights[0][r]
        if(value != 0.0):
            all_vals.append((cur_idx,value)) 
    else:
        cur_fact = factors[dim_no]
        for i in range(len(cur_fact)): # go through all rows
            cp = np.ndarray.copy(cur_prod);
            for r in range(rank): # go through each rank
                cp[r] *= cur_fact[i][r]
            expand(factors, weights, dim_no + 1, cur_idx + [i], cp, all_vals, rank)
            
def rebuild_sp_tensor_from_factors(kruskal_tensor, dimensions, rank):
    factors = kruskal_tensor[1]
    weights = kruskal_tensor[0]
    all_values = []
    expand(factors, weights, 0, [], np.ones(rank), all_values, rank)
    indices = [a[0] for a in all_values]
    values = [a[1] for a in all_values]
    shape = dimensions
    st = tf.SparseTensor(indices=indices, values=values, dense_shape=shape)
    return st

In [93]:
def generate_random_factors(dimensions, rank, d = 0.9):
    factors = [sparse.random(dim,rank,density=d).A for dim in dimensions]
    return factors

def expand_random_factors(factors, dim_no, cur_idx, cur_prod, all_vals, rank):
    #this method just writes to all values, so all values needs to be saved somewhere
    if dim_no == len(factors):
        value = sum(cur_prod) * (3.16**dim_no) # root 10. makes the numbers closer to [0-1]
        if(value != 0.0):
            all_vals.append((cur_idx,value)) 
    else:
        cur_fact = factors[dim_no]
        for i in range(len(cur_fact)):
            cp = np.ndarray.copy(cur_prod);
            for r in range(rank):
                cp[r] *= cur_fact[i][r]
            expand_random_factors(factors, dim_no + 1, cur_idx + [i], cp, all_vals, rank)
            
def generate_decomposable_sp_tensor(dimensions, rank):
    factors = generate_random_factors(dimensions, rank)
    all_values = []
    expand_random_factors(factors, 0, [], np.ones(rank), all_values, rank)
    indices = [a[0] for a in all_values]
    values = [a[1] for a in all_values]
    shape = dimensions
    st = tf.SparseTensor(indices=indices, values=values, dense_shape=shape)
    return st

def generate_random_sp_tensor(dimensions, d = 0.2):
    nd = len(dimensions)
    num_items = min(100000 , (int)(np.prod(dimensions) * d))    
    idxs = set()
    
    for i in range(num_items):
        rand = np.random.rand(nd) #gives us a random index
        index = tuple(np.trunc(np.multiply(rand,dimensions)).astype(int))
        idxs.add(index)
        
    indices = list(idxs)
    values = np.random.rand(len(indices))
    indices.sort()
    st = tf.SparseTensor(indices=indices, values=values, dense_shape=dimensions)
    return st

def generate_sp_tensor(dimensions, rank, d = 0.2, decomposable=True):
    if decomposable:
        return generate_decomposable_sp_tensor(dimensions, rank)
    else:
        return generate_random_sp_tensor(dimensions,d)

In [94]:
def tensor_norm(st):
    return (sum([x**2 for x in st.values.numpy()])**0.5)

def difference_frobenius_norm(spt1, spt2):
    idx1 = [tuple(s) for s in spt1.indices.numpy()]
    idx2 = [tuple(s) for s in spt2.indices.numpy()]
    val1 = spt1.values.numpy()
    val2 = spt2.values.numpy() 
    s1 = {idx1[i]:val1[i] for i in range(len(idx1))}
    s2 = {idx2[i]:val2[i] for i in range(len(idx2))}
    sum_sq = 0;
    for i in idx1:
        if i in idx2:
            sum_sq += (s1[i] - s2[i]) ** 2
        else:
            sum_sq += s1[i] ** 2
    for i in idx2:
        if i in idx1:
            sum_sq += 0
        else:
            sum_sq += s2[i] ** 2
    return sum_sq ** 0.5

In [100]:
shape1 = [4,7,2,9]
rank = 2
st1 = generate_sp_tensor(shape1, rank)
to_dense(st1)

<tf.Tensor: id=5358, shape=(4, 7, 2, 9), dtype=float64, numpy=
array([[[[2.62164312e+01, 2.62281191e+01, 4.78019905e+01,
          1.22187816e+01, 2.91795392e+01, 5.97670794e+00,
          4.76784049e-01, 3.06238330e+01, 7.79367895e+00],
         [1.10210294e+01, 1.02842103e+01, 2.26325576e+01,
          3.71694537e+00, 1.43367667e+01, 1.89105059e+00,
          0.00000000e+00, 1.49239395e+01, 3.81426521e+00]],

        [[8.06806869e+00, 7.52867194e+00, 1.65684186e+01,
          2.72103171e+00, 1.04953915e+01, 1.38436488e+00,
          0.00000000e+00, 1.09252380e+01, 2.79227580e+00],
         [3.96407968e+00, 3.69905818e+00, 8.14055187e+00,
          1.33692299e+00, 5.15669485e+00, 6.80179224e-01,
          0.00000000e+00, 5.36789108e+00, 1.37192731e+00]],

        [[1.01962093e+01, 9.97733406e+00, 1.93556187e+01,
          4.32455815e+00, 1.19721757e+01, 2.13728962e+00,
          1.25059362e-01, 1.25278759e+01, 3.19317808e+00],
         [4.52185687e+00, 4.21954476e+00, 9.28599155e+00,


In [101]:
cpd = cp_als(st1, rank, 100)
cpd

(array([[6.26390985e+00, 1.99340575e+23]]), [array([[ 0.        , -0.77394098],
         [ 0.        , -0.08939727],
         [ 0.        , -0.19918184],
         [ 0.        , -0.59443258]]), array([[0.        , 0.55047128],
         [0.        , 0.18402502],
         [0.        , 0.22010656],
         [0.        , 0.64385542],
         [0.        , 0.42216433],
         [0.        , 0.14621578],
         [0.        , 0.02275286]]), array([[0.00000000e+00, 4.14426642e-22],
         [0.00000000e+00, 1.86918421e-22]]), array([[ 0.        , -0.33479691],
         [ 0.        , -0.32543338],
         [ 0.        , -0.64299578],
         [ 0.        , -0.13783263],
         [ 0.        , -0.39918633],
         [ 0.        , -0.06835506],
         [ 0.        , -0.00351819],
         [ 0.        , -0.41737432],
         [ 0.        , -0.1064279 ]])])

In [102]:
rebuilt = rebuild_sp_tensor_from_factors(cpd, shape1, rank)
to_dense(rebuilt)

<tf.Tensor: id=6165, shape=(4, 7, 2, 9), dtype=float64, numpy=
array([[[[1.17833103e+01, 1.14537573e+01, 2.26304922e+01,
          4.85107417e+00, 1.40495216e+01, 2.40578355e+00,
          1.23824193e-01, 1.46896554e+01, 3.74577230e+00],
         [5.31461430e+00, 5.16597635e+00, 1.02070076e+01,
          2.18797498e+00, 6.33674123e+00, 1.08507807e+00,
          5.58482982e-02, 6.62546012e+00, 1.68945182e+00]],

        [[3.93921351e+00, 3.82904246e+00, 7.56547508e+00,
          1.62173586e+00, 4.69681810e+00, 8.04264236e-01,
          4.13949834e-02, 4.91081770e+00, 1.25222849e+00],
         [1.77669940e+00, 1.72700907e+00, 3.41224842e+00,
          7.31449851e-01, 2.11840102e+00, 3.62746468e-01,
          1.86703367e-02, 2.21492104e+00, 5.64791322e-01]],

        [[4.71156993e+00, 4.57979779e+00, 9.04882785e+00,
          1.93970748e+00, 5.61771706e+00, 9.61955270e-01,
          4.95112435e-02, 5.87367529e+00, 1.49775128e+00],
         [2.12505453e+00, 2.06562147e+00, 4.08128349e+00,


In [103]:
difference_frobenius_norm(st1, rebuilt)

107.61174044672632

In [104]:
tensor_norm(st1)

197.82343970856815